# Elastic constants

For the source code, see [elastic](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.elastic.elastic_workgraph).

## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.ase.espresso.elastic import elastic_workgraph

task = elastic_workgraph.task()
task.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.elastic import elastic_workgraph
from aiida import load_profile

load_profile()

wg = elastic_workgraph()
wg.to_html()

## Example: Aluminum


### Prepare the inputs and submit the workflow


In [10]:
from ase.build import bulk
from aiida import load_profile
from workgraph_collections.ase.espresso.elastic import elastic_workgraph

load_profile()

# create input structure
atoms = bulk("Al")


metadata = {
    "options": {
        'prepend_text' : """
eval "$(conda shell.posix hook)"
conda activate aiida
export OMP_NUM_THREADS=1
""",
    }
}

pseudopotentials = {"Al": "Al.pbe-nl-rrkjus_psl.1.0.0.UPF"}
# pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
pseudo_dir = "/home/wang_x3/datas/pseudos/psl"

# we need use high conv_thr to get accurate elastic constants
input_data = {
    "control": {"calculation": "relax", 
                "tstress": True,
                "tprnfor": True,
                "verbosity": "high",
                "etot_conv_thr": 1e-5,
                "forc_conv_thr": 1.0e-5,
                },
    "system": {"ecutwfc": 40, "ecutrho": 320,
               "occupations": "smearing",
               "smearing": "gaussian",
                "degauss": 0.01,
                },
    "electrons": {"conv_thr": 1e-11}
}
#------------------------- Set the inputs -------------------------
wg = elastic_workgraph(atoms=atoms,
                   computer="localhost",
                   command="mpirun -np 4 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   kpts=(30, 30, 30),
                   metadata=metadata)
#------------------------- Submit the calculation -------------------
wg.submit(wait=True, timeout=500)


WorkGraph process created, PK: 20671


<WorkChainNode: uuid: 2502d582-697a-4683-8839-cc8cace66188 (pk: 20671) (aiida_workgraph.engine.workgraph.WorkGraphEngine)>

In [11]:
#------------------------- Print the output -------------------------
import numpy as np
elastic_constants = wg.tasks["fit_elastic"].outputs["result"].value.value
# 1 eV/Angstrom3 = 160.21766208 GPa
elastic_constants = elastic_constants * 160.21766208
print("Elastic constants: ")
print(np.round(elastic_constants, 2))


Elastic constants: 
[[117.33  57.59  57.59  -0.    -4.2    4.2 ]
 [ 57.59 117.33  57.59   4.2    0.    -4.2 ]
 [ 57.59  57.59 117.33  -4.2    4.2   -0.  ]
 [  0.     0.    -0.    38.14  -0.    -0.  ]
 [  0.     0.    -0.    -0.    38.14  -0.  ]
 [ -0.    -0.    -0.    -0.    -0.    38.14]]


The calcualted elastic constants of aluminum are C<sub>11</sub>=117.3 GPa, C<sub>12</sub>=57.6 GPa, and C<sub>44</sub>=38.1 GPa, compared to the values reported by [experiment](https://pubs.aip.org/aip/jap/article/35/6/1825/133322/Elastic-Constants-of-Aluminum): C<sub>11</sub>=116.3 GPa, C<sub>12</sub>=64.8 GPa, and C<sub>44</sub>=30.9 GPa.